# INSTRUCTION TO RUN THE CODE:

# TASK - 1: mBert:



1.   **Training and Testing**



*   **Loading Dataset** : Load the dataset for training and give path of the dataset at 'Data Processing block' for reading the dataset.
*   **Testing Dataset** : For testing yor csv file you have to load your test_dataset.csv file and provide its path to the block 'preprocessing of test Dataset'.


*   For removing stopword you have to upload one extra file final_stopwords.tx and provide its correct path

2.   **Loading trained Model and Test:**


*   You can test your test csv by just loading the already saved trained model mbert_model.pth that reduce training time

# TASK - 2: MURiL:



1.   **Training and Testing**



*   **Loading Dataset** : Load the dataset for training and give path of the dataset at 'Data Processing block' for reading the dataset.
*   **Testing Dataset** : For testing yor csv file you have to load your test.csv file and provide its path to the block 'preprocessing of test Dataset'.


*   For removing stopword you have to upload one extra file final_stopwords.tx and provide its correct path



2.   **Loading trained Model and Test:**


*   You can test your test csv by just loading the already saved trained model muril_model.pth that reduce training time




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Task - 3

# Install all necessary libraries

In [ ]:
import locale

In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import locale
print(locale.getdefaultlocale())
print(locale.getpreferredencoding())

('en_US', 'UTF-8')
UTF-8


In [ ]:
!pip install torch torchvision transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

import collections
from collections import Counter

In [ ]:
# Loading the dataset
full_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_train_val.csv')
# df = full_df[["text"]]
df = full_df
# df["text"] = df["text"].astype(str)
full_df.head()

,label,text
0,0,भीड़ में बहुत हब्सी मिलेंगे
1,0,साले बेवकूफ अपनी मां मक्खियां तो हटा दें
2,0,बुर देदो तो मुह में लंड ले लो तो
3,0,कुत्ता वहा है चिल्ला तू क्यों रहा है
4,1,चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok


In [ ]:
!pip install indic_transliteration indicnlp indic-nlp-library indic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 15.4 MB/s eta 0:00:00
  Created wheel for indic: filename=indic-0.1.2-py3-none-any.whl size=10089 sha256=bbc3d8866ed0af608ce1923e03a034a61649855e973f2a8ca4a5c5e3b2f01f04
  Stored in directory: /root/.cache/pip/wheels/99/46/ae/5111371e0c7d7160c689f9f6cee321acd45a4e32d4c419dd4a
Successfully built indic


In [ ]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
# stopword list

with open('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/final_stopwords.txt', 'r', encoding="utf8") as file:
    content = file.readlines()

stopwords_hi = [line.strip() for line in content]

# stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
# punctuations = ['nn','n', '।','/', '`', '+', '\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}','|','"']

# punctuations = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"]

punctuations = [".", ",", "!", "?", ":", ";", "-", "—", "(", ")", "[", "]", "{", "}", "/", "\\", "|", "@", "#", "$", "%", "^", "&", "*", "+", "=", "<", ">", "_", "~", "`", "'", "\"", "«", "»", "“", "”", "‘", "’", "‹", "›", "„", "‟", "‛", "‟", "‘", "’", "‚", "‛", "‟", "‹", "›", "〈", "〉", "《", "》", "「", "」", "『", "』", "【", "】"]



to_be_removed = stopwords_hi + punctuations + stopwords_en

In [ ]:
# removing all stopword

for i in range(len(df)):
    df['text'][i]=[ele for ele in df['text'][i] if ele not in (to_be_removed)]
# count_length()
df.tail(10)

,label,text
20173,0,"[ठोक, तू, अच्छा, ना]"
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂..."
20175,0,"[तेरी, माँ, कसम]"
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह..."
20177,1,"[तीसरा, नाम्, कल्लू]"
20178,0,"[छोटे, क्यु, पहनते, जरूरत]"
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]"
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]"
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]"
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा..."


In [ ]:
# Converting list to string 

# Create a new column called 'processed_text'
df['processed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(df)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(df['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    df.loc[i, 'processed_text'] = text_string

# Call the count_length() function (assuming it's defined somewhere)
# count_length()

df.tail(10)


,label,text,processed_text
20173,0,"[ठोक, तू, अच्छा, ना]",ठोक तू अच्छा ना
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...",💥💥💥 गुड नाइट जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...
20175,0,"[तेरी, माँ, कसम]",तेरी माँ कसम
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह...",जीवन जीने के तरीके मायने के चमत्कार नही दुसरे ...
20177,1,"[तीसरा, नाम्, कल्लू]",तीसरा नाम् कल्लू
20178,0,"[छोटे, क्यु, पहनते, जरूरत]",छोटे क्यु पहनते जरूरत
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]",कैसी सोना हमसे दोस्ती करोगी
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]",पता चलता चुनाव आ है🙄🙄🙄
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]",खेत जोते किसान भैंस के साथो
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा...",काश आगर इतना दिमाग पढाई लगायें शायद जाकर डॉक्ट...


# Preprocessing Test datase

In [ ]:
# Loading the dataset
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_test.csv')
# df = full_df[["text"]]
tdf = test_df
# df["text"] = df["text"].astype(str)
tdf.head()

,label,text
0,0,मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सो...
1,1,और दिवाली में भी पूरा देश पड़ाका नहीं फोडात
2,1,कुत्ता बिल्ली पाल लेना मगर गलत फहमी कभी नहीं।
3,0,तेरी गांड में प्याज काट देगा गुज्जर भोसड़ी के
4,1,बंगाली साड़ी ऐसे नहीं पहना जाता है दीदी


In [ ]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
tdf['text'] = tdf['text'].apply(lambda x: tokenization(x))

# removing all stopword

for i in range(len(tdf)):
    tdf['text'][i]=[ele for ele in tdf['text'][i] if ele not in (to_be_removed)]
# count_length()
# tdf.tail(10)

# Converting list to string 

# Create a new column called 'processed_text'
tdf['tprocessed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(tdf)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(tdf['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    tdf.loc[i, 'tprocessed_text'] = text_string

# Call the count_length() function (assuming it's defined somewhere)
# count_length()

tdf.tail(10)


,label,text,tprocessed_text
6718,0,"[चूतियो, कमी, नहीं, बढ़कर, चुतिय]",चूतियो कमी नहीं बढ़कर चुतिय
6719,0,"[प्यार❤, व्यार, 💔साला, सब, धोखा, 😡है, बिना, जि...",प्यार❤ व्यार 💔साला सब धोखा 😡है बिना जिंदगी 🌎जी...
6720,0,"[नंगी, होकर, वीडियो, बनाओ, अच्छी, लगोगी]",नंगी होकर वीडियो बनाओ अच्छी लगोगी
6721,0,"[सही, तरीके, कपड़े, दिखाने, के, मेरे, पास, कपड़ा]",सही तरीके कपड़े दिखाने के मेरे पास कपड़ा
6722,1,"[बीजेपी, देन, मंदिर, सब, मंदिर, रुकवाने, कसर, ...",बीजेपी देन मंदिर सब मंदिर रुकवाने कसर नहीं छोड...
6723,0,"[आरे, पागल, लवडे, समज, आये, वो, ना]",आरे पागल लवडे समज आये वो ना
6724,0,"[शेर, नज़र, अंदाज़, ।, कमजोरी, मत, समझ, लेना, तु...",शेर नज़र अंदाज़ । कमजोरी मत समझ लेना तुम्हारे कौ...
6725,0,"[लाल, चड्डी, के, लाल, होगा]",लाल चड्डी के लाल होगा
6726,1,[],
6727,0,"[पोरीच्या, फोटो, काय, महाराजांचे, नाटक, रे, मा...",पोरीच्या फोटो काय महाराजांचे नाटक रे मादरचोत


In [ ]:
device='cuda'

# TASK - 1

# Implementation of mBERT for Training

In [ ]:
from sklearn.model_selection import train_test_split

traindf, valdf = train_test_split(df,train_size=0.8)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.to(device)
sentences = list(df['processed_text'])
labels = list(df['label'])

# Tokenize the input sentences
encoded_dict = tokenizer.batch_encode_plus(
                sentences,                      # List of input sentences
                add_special_tokens = True,      # Add [CLS] and [SEP] tokens
                max_length = 64,               # Truncate or pad to this length
                padding = 'max_length',        # Pad to the max_length
                return_attention_mask = True,  # Return attention masks
                return_tensors = 'pt',         # Return PyTorch tensors
                truncation=True)

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']

# Convert the labels to a tensor
labels = torch.tensor(labels)


train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

epochs = 10

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Do Not Run below cell if you do not want to train again


In [ ]:


for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch in train_loader:
        # Load the input data and labels to the device (CPU or GPU)
        input_ids, attention_masks, labels = tuple(t.to(device) for t in batch)

        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)

        # Calculate the loss
        loss = criterion(outputs.logits, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
    
    train_accuracy = correct / total
    train_loss = running_loss / len(train_loader)
    
    # Evaluate on test set
    model.eval()
    with torch.no_grad():
      running_loss = 0.0
      total = 0
      correct = 0
      for data in test_loader:
          input_ids, attention_masks, labels = [t.to(device) for t in data]
          outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
          
          loss = criterion(outputs.logits, labels)

          running_loss += loss.item()

          _, predicted = torch.max(outputs.logits.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    test_loss = running_loss / len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train loss: {train_loss:.4f} Train accuracy: {train_accuracy:.4f}")
    print(f"Test loss: {test_loss:.4f} Test accuracy: {test_accuracy:.4f}")
    print("="*50)



# Saving Trained model of mBERT for testing Purpose

In [ ]:
# Save the model
torch.save(model, 'mbert_model.pth')

# Loading mBERT for Testing on test.csv



In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
# Load the saved model
model_path = '/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/mbert_model.pth'
# Instantiate the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the saved weights
state_dict = torch.load(model_path)
# print(state_dict)

# Load the state_dict into the model
model.load_state_dict(state_dict.state_dict())
model.to(device)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Same steps of validation from previous code

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# TRAINED mBERT MODEL ACCURACY REPORT

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Set the model to evaluation mode
model.eval()

# Initialize empty lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test data and generate predictions
for data in test_loader:
    input_ids, attention_masks, labels = [t.to(device) for t in data]
    outputs = model(input_ids, attention_mask=attention_masks)
    _, predicted = torch.max(outputs.logits.data, 1)
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

# Calculate accuracy score
accuracy = accuracy_score(true_labels, predicted_labels)

# Print accuracy report
print("Accuracy:", accuracy)
print(classification_report(true_labels, predicted_labels))


Accuracy: 0.7849888531087441
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      2140
           1       0.78      0.76      0.77      1897

    accuracy                           0.78      4037
   macro avg       0.78      0.78      0.78      4037
weighted avg       0.78      0.78      0.78      4037



# Testing using trained model of mBERT and printing report

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader

sentences = list(tdf['tprocessed_text'])
labels = list(tdf['label'])

# Tokenize the input sentences
encoded_dict = tokenizer.batch_encode_plus(
                sentences,                      # List of input sentences
                add_special_tokens = True,      # Add [CLS] and [SEP] tokens
                max_length = 64,               # Truncate or pad to this length
                padding = 'max_length',        # Pad to the max_length
                return_attention_mask = True,  # Return attention masks
                return_tensors = 'pt',         # Return PyTorch tensors
                truncation=True)

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']

# Convert the labels to a tensor
labels = torch.tensor(labels)

# from sklearn.model_selection import train_test_split

# train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
# train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

# from torch.utils.data import TensorDataset, DataLoader

# train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(input_ids, attention_masks, labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)  
    
    
    
    
# Evaluate on test set
from sklearn.metrics import classification_report

model.eval()
with torch.no_grad():
  predictions = []
  true_labels = []
  for data in test_loader:
      input_ids, attention_masks, labels = [t.to(device) for t in data]
      outputs = model(input_ids, attention_mask=attention_masks)
          
      _, predicted = torch.max(outputs.logits.data, 1)
      predictions.extend(predicted.cpu().numpy())
      true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.78      0.81      0.80      3496
           1       0.79      0.75      0.77      3232

    accuracy                           0.79      6728
   macro avg       0.79      0.78      0.79      6728
weighted avg       0.79      0.79      0.79      6728



# TASK - 2

# Implementation of MuRil

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "google/muril-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sentences = list(df['processed_text'])
labels = list(df['label'])

# Tokenize the input sentences
encoded_dict = tokenizer.batch_encode_plus(
                sentences,                      # List of input sentences
                add_special_tokens = True,      # Add [CLS] and [SEP] tokens
                max_length = 64,               # Truncate or pad to this length
                padding = 'max_length',        # Pad to the max_length
                return_attention_mask = True,  # Return attention masks
                return_tensors = 'pt',         # Return PyTorch tensors
                truncation=True)

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']

# Convert the labels to a tensor
labels = torch.tensor(labels)

from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

epochs = 1

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

# Do Not Run below cell if you do not want to train again

In [ ]:


for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch in train_loader:
        # Load the input data and labels to the device (CPU or GPU)
        input_ids, attention_masks, labels = tuple(t.to(device) for t in batch)

        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)

        # Calculate the loss
        loss = criterion(outputs.logits, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
    
    train_accuracy = correct / total
    train_loss = running_loss / len(train_loader)
    
    # Evaluate on test set
    model.eval()
    with torch.no_grad():
      running_loss = 0.0
      total = 0
      correct = 0
      for data in test_loader:
          input_ids, attention_masks, labels = [t.to(device) for t in data]
          outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
          
          loss = criterion(outputs.logits, labels)

          running_loss += loss.item()

          _, predicted = torch.max(outputs.logits.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      test_accuracy = correct / total
      test_loss = running_loss / len(test_loader)

      print(f"Epoch {epoch+1}/{epochs}")
      print(f"Train loss: {train_loss:.4f} Train accuracy: {train_accuracy:.4f}")
      print(f"Test loss: {test_loss:.4f} Test accuracy: {test_accuracy:.4f}")
      print("="*50)
          


# Saving Trained model of MURiL fro testing Purpose

In [ ]:
import torch

# Save the model to a file
torch.save(model, 'muril_model.pth')


# Loading saved MURiL for Testing

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load the saved model
model_path = '/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/muril_model.pth'
# Instantiate the model
model = AutoModelForSequenceClassification.from_pretrained('google/muril-base-cased')

# Load the saved weights
state_dict = torch.load(model_path)
# print(state_dict)

# Load the state_dict into the model
model.load_state_dict(state_dict.state_dict())
model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

# Same steps of validation from previous code

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

# TRAINED MODEL ACCURACY REPORT OF MURiL

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Set the model to evaluation mode
model.eval()

# Initialize empty lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test data and generate predictions
for data in test_loader:
    input_ids, attention_masks, labels = [t.to(device) for t in data]
    outputs = model(input_ids, attention_mask=attention_masks)
    _, predicted = torch.max(outputs.logits.data, 1)
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

# Calculate accuracy score
accuracy = accuracy_score(true_labels, predicted_labels)

# Print accuracy report
print("Accuracy:", accuracy)
print(classification_report(true_labels, predicted_labels))


Accuracy: 0.8397324746098588
              precision    recall  f1-score   support

           0       0.86      0.83      0.85      2140
           1       0.82      0.85      0.83      1897

    accuracy                           0.84      4037
   macro avg       0.84      0.84      0.84      4037
weighted avg       0.84      0.84      0.84      4037



# Testing using trained model of MURiL and printing report


In [ ]:
sentences = list(tdf['tprocessed_text'])
labels = list(tdf['label'])

# Tokenize the input sentences
encoded_dict = tokenizer.batch_encode_plus(
                sentences,                      # List of input sentences
                add_special_tokens = True,      # Add [CLS] and [SEP] tokens
                max_length = 64,               # Truncate or pad to this length
                padding = 'max_length',        # Pad to the max_length
                return_attention_mask = True,  # Return attention masks
                return_tensors = 'pt',         # Return PyTorch tensors
                truncation=True)

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']

# Convert the labels to a tensor
labels = torch.tensor(labels)
test_dataset = TensorDataset(input_ids, attention_masks, labels)

# test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)  
    
# Evaluate on test set
from sklearn.metrics import classification_report

model.eval()
with torch.no_grad():
  predictions = []
  true_labels = []
  for data in test_loader:
      input_ids, attention_masks, labels = [t.to(device) for t in data]
      outputs = model(input_ids, attention_mask=attention_masks)
          
      _, predicted = torch.max(outputs.logits.data, 1)
      predictions.extend(predicted.cpu().numpy())
      true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))



              precision    recall  f1-score   support

           0       0.86      0.83      0.84      3496
           1       0.82      0.85      0.84      3232

    accuracy                           0.84      6728
   macro avg       0.84      0.84      0.84      6728
weighted avg       0.84      0.84      0.84      6728



# Printing Intersection points

In [ ]:
def print_intersection_points(train_df, val_df, new_testdf):
  train_com_count = 0
  val_com_count = 0
  for sentence in train_df['text']:
    for new_sentence in new_testdf['text']:
      if sentence==new_sentence:
        train_com_count += 1

  for sentence in val_df['text']:
    for new_sentence in new_testdf['text']:
      if sentence==new_sentence:
        val_com_count += 1
  
  # print("Intersection points for dataset used for testing and training ", modelname)
  print("Common points with training dataset: ",train_com_count)
  print("Common points with validation dataset: ",val_com_count)

In [ ]:
print_intersection_points(traindf,valdf,test_df)

Common points with training dataset:  161
Common points with validation dataset:  61
